# RDD API Spark

Для начала давайте поработаем с RDD API. Начем с классической задачи **wordcount** (рассмотренной в лекции)

In [1]:
from nltk.tokenize import RegexpTokenizer

Считаем данные в rdd

In [2]:
rdd = sc.textFile("wiki_math/*")

Посмотрим, что у нас там в данных: 

In [3]:
rdd.take(1)

['Ложкин, Сергей Андреевич В Википедии есть статьи о других людях с такой фамилией, см. Ложкин; Ложкин, Сергей. Ложкин Сергей Андреевич    Дата рождения  9 марта 1951(1951-03-09) (67 лет)   Место рождения  Киров, СССР   Научная сфера  дискретная математика   Место работы  ВМК МГУ   Альма-матер  МГУ (1973)   Учёная степень  доктор физико-математических наук\xa0(1998)   Учёное звание  профессор (2003),заслуженный профессор (2009)   Научный руководитель  О.\xa0Б.\xa0Лупанов   Известные ученики  М. С. Шуплецов,Б. Р. Данилов,Е. Л. Довгалюк,В. А. Коноводов,В. В. Жуков,Л. И. Высоцкий   Награды и\xa0премии     Сайт  Страница на сайте кафедры ВМК МГУ   Ло́жкин Серге́й Андре́евич (род. 1951)\xa0— российский математик, доктор физико-математических наук, профессор, зам. декана по научной работе и финансам факультета ВМК МГУ.  Содержание  1 Биография 2 Научная деятельность 3 Педагогическая деятельность 4 Избранные работы 5 Примечания 6 Ссылки   Биография[править | править код] В 1968 году окончил с

Функция map будет возвращать список пар (слово, 1) для каждого слова в тексте

In [4]:
tokenizer = RegexpTokenizer("[а-яa-z]+")
def map_func(text):
    result = []
    for token in tokenizer.tokenize(text.lower()):
        result.append((token, 1))
    return result

In [5]:
map_func('Куб принца Руперта')

[('куб', 1), ('принца', 1), ('руперта', 1)]

Посчитаем количество раз, которое встретилось каждой слово в наших текстах. Для этого применим подход MapReduce

In [6]:
word_counts_rdd = rdd.flatMap(map_func).reduceByKey(lambda x, y: x+y)

Соберем данные локально

In [7]:
word_counts = word_counts_rdd.collectAsMap()

In [8]:
word_counts['а']

1525

При выполнении задания вам может оказаться полезной метод .mapValues(func)

Если датасет представлен парами ключ-значение, то mapValues применяет func только к значению, а ключ оставляет без изменений. Например, применим функцию, которая делит все значения пополам:

In [9]:
print("original:", word_counts_rdd.take(5))
print("mapValues:", word_counts_rdd.mapValues(lambda x: x / 2).take(5))

original: [('первых', 47), ('brief', 1), ('удвоение', 3), ('речевой', 2), ('свести', 11)]
mapValues: [('первых', 23.5), ('brief', 0.5), ('удвоение', 1.5), ('речевой', 1.0), ('свести', 5.5)]


# Задание 1

При помощи функций rdd .map(), .flatMap(), .reduceByKey(), .count(), .mapValues реализуйте подсчет IDF для каждого слова в нашем датасете. 

Напоминаем, что IDF рассчитывается как **math.log(N/t)** где N - количество документов в наборе, а t - количество документов, в которых встретилось слово t

`
Обратите внимание, что после того как вы вполнили функции .flatMap() и .reduceByKey(), к результату снова можно применить например .mapValues()



In [11]:
rdd_count = rdd.count()
rdd_count

655

In [28]:
import math

#Цепочка вызовов Для получения rdd вида [('слово1', 0.32423), ... ('словоN', IDF_N) ]
idfs_rdd = rdd.flatMap(lambda x: [(z, 1) for z in set(y[0] for y in map_func(x))]).reduceByKey(lambda x, y: x+y).map(lambda x: (x[0], math.log(rdd_count / x[1])))
idfs_rdd.take(5)

[('первых', 2.9582747110190906),
 ('brief', 6.484635235635252),
 ('удвоение', 5.386022946967143),
 ('речевой', 5.791488055075306),
 ('свести', 4.182050142641207)]

In [29]:
result = idfs_rdd.collectAsMap()

### Задание: 
Отправьте в учебную платформу результат IDF полученный для токена python 

In [30]:
print(result['python'])

4.086739962836881


# Dataframe API

Теперь давайте поработаем со spark Dataframe Api. Вашей задачей будет написать классификатор, который предсказывает пол молюска **морское ушко** по его разнообразным параметрам.

Первым шагом давайте загрузим данные в dataframe и посмотрим из чего они состоят:

In [31]:
data = spark.read.csv('data.csv', header=True, inferSchema=True)
data.show(5)

+---+------+--------+------+------------------+--------------+--------------+------------+----+
|Sex|Length|Diameter|Height|      Whole weight|Shucked weight|Viscera weight|Shell weight| Age|
+---+------+--------+------+------------------+--------------+--------------+------------+----+
|  0|  0.59|   0.455| 0.155|1.0659999999999998|         0.382|        0.2275|       0.415|21.5|
|  1|  0.31|   0.245| 0.095|              0.15|        0.0525|         0.034|       0.048| 8.5|
|  1| 0.545|    0.39| 0.135|            0.7835|        0.4225|        0.1815|       0.156| 8.5|
|  0|  0.41|    0.32| 0.115|             0.387|         0.165|        0.1005|      0.0985|12.5|
|  0|  0.41|   0.325| 0.105|            0.3635|         0.159|         0.077|        0.12|11.5|
+---+------+--------+------+------------------+--------------+--------------+------------+----+
only showing top 5 rows



Параметр **inferSchema**, которым мы воспользовались,  позволяет автоматически определять типы данных, и не преобразовывать их руками, как мы это делали в видео.

Также загрузим контрольные данные: 

In [32]:
control_data = spark.read.csv('control.csv', header=True, inferSchema=True)
control_data.show(4)

+------+--------+------+------------------+--------------+-------------------+--------------------+----+
|Length|Diameter|Height|      Whole weight|Shucked weight|     Viscera weight|        Shell weight| Age|
+------+--------+------+------------------+--------------+-------------------+--------------------+----+
|  0.63|   0.505| 0.175|1.2209999999999999|         0.555|              0.252|                0.34|13.5|
|  0.59|    0.47| 0.145|            0.9235|        0.4545|0.17300000000000001|               0.254|10.5|
|  0.69|   0.555| 0.205|            1.8165|        0.7785|             0.4395|               0.515|20.5|
| 0.345|    0.27|  0.09|             0.195|         0.078|             0.0455|0.059000000000000004|10.5|
+------+--------+------+------------------+--------------+-------------------+--------------------+----+
only showing top 4 rows



Как видитие, контрольные данные отличаются от тренировочных тем, что в них отсутсвует 

## Задание

Напишите классификатор, который предсказывает пол (переменная Sex) молюска, используя библиотеку SparkML. Для этого, воспользуйтесь примером из видео лекции.


В примере мы построим совсем простой классификатор, использующий только две фичи. При обучении своей модели **не забудте добавить и другие фичи в модель**. 

В примере мы не реализуем разделение тренировочной выборки на train и test, но рекомендум вам реализовать его самостоятельно, воспользовавшись примером из лекции. 

Рекомендуем вам попробовать разные алгоритмы классификации из библиотеки SparkML (https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#classification), а также попробовать выполнить поиск гиперпараметров для моделей. 

Начать рекомендуем с уже знакомых вам моделей **RandomForestClassifier** и  **GBTClassifier**



In [38]:
from pyspark.ml.feature import VectorAssembler


#для примера используем только две фичи, Diameter и Age
assembler = VectorAssembler(inputCols=['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Age'], outputCol='features')


#Создадим новый датафрейм, содержащий отдельную колонку с фичами
data_transformed = assembler.transform(data)
control_data_transformed = assembler.transform(control_data)


In [39]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol="Sex", featuresCol="features")
model = classifier.fit(data_transformed)

#### Выполним преобразование данных и подготовим их для отправки в EdX

Скопируйте строку с предсказанными вероятностями и вставьте в поле ввода второго задания, чтобы узнать свой результат. 

In [40]:
predictions = model.transform(control_data_transformed).collect()
result = [str(prediction["probability"][1]) for prediction in predictions]
print(",".join(result))

0.5489913544668588,0.5489913544668588,0.42105263157894735,0.8867924528301887,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.7142857142857143,0.5489913544668588,0.5489913544668588,0.12903225806451613,0.42105263157894735,0.7142857142857143,0.6851851851851852,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.42105263157894735,0.12903225806451613,0.42105263157894735,0.5489913544668588,0.5489913544668588,0.12903225806451613,0.5489913544668588,0.42105263157894735,0.5489913544668588,0.42105263157894735,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.42105263157894735,0.3424657534246575,0.7142857142857143,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.5489913544668588,0.42105263157894735,0.8867924528301887,0.42105263157894735,0.5489913544668588,

### Оценивание Задания

Ваша оценка будет зависеть от метрики ROC_AUC на контрольных данных. 

Количество баллов будет рассчитываться по следующей функции: 
Мы не гарантируем того, что можно набрать полный балл, но ваша задача набрать максимум сколько вы сможете :)


In [36]:
def score_func(auc):
    if auc < 0.55:
        return 0

    if auc >= 0.6:
        return 1.0

    return int((1 - (0.6 - auc) / 0.05)/ 2 * 100) / 100 + 0.5

Вот несколько примеров того, сколько баллов вы наберете в зависимости от метрики AUC:

In [37]:
auc_scores = [0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.599, 0.6]
for auc in auc_scores:
    print("Если ваш AUC равен {} вы наберете {} баллов".format(auc, score_func(auc)))

Если ваш AUC равен 0.54 вы наберете 0 баллов
Если ваш AUC равен 0.55 вы наберете 0.5 баллов
Если ваш AUC равен 0.56 вы наберете 0.6 баллов
Если ваш AUC равен 0.57 вы наберете 0.69 баллов
Если ваш AUC равен 0.58 вы наберете 0.79 баллов
Если ваш AUC равен 0.59 вы наберете 0.89 баллов
Если ваш AUC равен 0.599 вы наберете 0.99 баллов
Если ваш AUC равен 0.6 вы наберете 1.0 баллов
